In [1]:
!pip install 'accelerate>=0.26.0' torch transformers


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from huggingface_hub import login
import os

# Login
login()

In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import logging

logging.basicConfig(level=logging.INFO)

class Agent:
    def __init__(self, name, role, model_name=None, model=None, tokenizer=None, device=None, temperature=0.7, top_p=0.9, max_new_tokens=512):
        self.name = name
        self.role = role  # Solver, Critic, or Judge
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.max_new_tokens = max_new_tokens
        self.temperature = temperature
        self.top_p = top_p

        if model is not None and tokenizer is not None:
            # Use the provided model and tokenizer
            self.model = model.to(self.device)
            self.tokenizer = tokenizer
        elif model_name is not None:
            # Load tokenizer and model with error handling
            try:
                self.tokenizer = AutoTokenizer.from_pretrained(model_name)
                self.model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    torch_dtype=torch.float16 if self.device.type == 'cuda' else torch.float32
                ).to(self.device)
            except Exception as e:
                logging.error(f"Failed to load model {model_name}: {e}")
                raise
        else:
            raise ValueError("Either model and tokenizer or model_name must be provided")

        # Set up the pipeline with flexible parameters
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if self.device.type == 'cuda' else -1,
            max_new_tokens=self.max_new_tokens,
            do_sample=True,
            temperature=self.temperature,
            top_p=self.top_p,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
        )
        self.history = []
        self.score = 1.0

    def generate_response(self, messages):
        """
        Generate a response from the model based on the input messages.
        """
        prompt = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])
        prompt += f"\n{self.role}:"
        try:
            response = self.pipeline(prompt)[0]['generated_text']
            # Extract the assistant's response
            response = response[len(prompt):].strip()
        except Exception as e:
            logging.error(f"Error generating response: {e}")
            return "Sorry, I couldn't generate a response."
        return response


def construct_message(agent, previous_responses, question):
    """
    Construct a message for the agent based on its role.
    """
    if agent.role == 'Solver':
        # Solver must provide a detailed solution to the question
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Solver. Your task is to solve the following problem in detail, providing clear and complete explanations, including any mathematical proofs and examples where appropriate."},
            {'role': 'user', 'content': f"The problem to solve is: '{question}'. Please provide your detailed solution before anyone else responds."}
        ]
    elif agent.role == 'Critic':
        # Critic must provide a detailed critique of the solution
        responses_summary = "\n".join([f"Solver's solution: {resp}" for resp in previous_responses])
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Critic. Provide a detailed critique of the solution provided by the Solver, pointing out any errors or areas for improvement, and offering suggestions for correction."},
            {'role': 'user', 'content': f"The Solver has presented the following solution:\n{responses_summary}\nProvide your comprehensive critique."}
        ]
    elif agent.role == 'Judge':
        # Judge must evaluate both the solution and the critique
        solver_response, critic_response = previous_responses
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Judge. Evaluate the solution provided by the Solver and the critique provided by the Critic in detail. Assess the correctness of the solution, the validity of the critique, and provide a final verdict with explanations."},
            {'role': 'user', 'content': f"Solver's solution:\n{solver_response}\n\nCritic's critique:\n{critic_response}\n\nProvide your detailed evaluation and final decision."}
        ]


def run_debate(agents, question, rounds=1):
    """
    Run a multi-agent debate where agents respond in a controlled sequential order.
    """
    solver = next(agent for agent in agents if agent.role == 'Solver')
    critic = next(agent for agent in agents if agent.role == 'Critic')
    judge = next(agent for agent in agents if agent.role == 'Judge')

    # Step 1: Solver provides the solution
    solver_messages = construct_message(solver, [], question)
    solver_response = solver.generate_response(solver_messages)
    print(f"{solver.name} (Solver):\n{solver_response}\n")

    # Step 2: Critic critiques the solution
    critic_messages = construct_message(critic, [solver_response], question)
    critic_response = critic.generate_response(critic_messages)
    print(f"{critic.name} (Critic):\n{critic_response}\n")

    # Step 3: Continue the debate if there are more rounds
    for round_num in range(2, rounds + 1):
        # Solver may refine the solution based on critique
        solver_messages = construct_message(solver, [critic_response], question)
        solver_response = solver.generate_response(solver_messages)
        print(f"Round {round_num} - {solver.name} (Solver):\n{solver_response}\n")

        # Critic responds with further critique
        critic_messages = construct_message(critic, [solver_response], question)
        critic_response = critic.generate_response(critic_messages)
        print(f"Round {round_num} - {critic.name} (Critic):\n{critic_response}\n")

    # Step 4: Judge evaluates the final responses
    judge_messages = construct_message(judge, [solver_response, critic_response], question)
    judge_response = judge.generate_response(judge_messages)
    print(f"{judge.name} (Judge):\n{judge_response}\n")

    # Return the final result decided by the judge
    return judge_response


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model name
model_name = 'meta-llama/Llama-3.1-8B-Instruct'

# Load the model and tokenizer once
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32
    ).to(device)
except Exception as e:
    logging.error(f"Failed to load model {model_name}: {e}")
    raise

# Create agents with the same model and tokenizer
agents = [
    Agent(name='Agent1', role='Solver', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent2', role='Critic', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent3', role='Judge', model=model, tokenizer=tokenizer, device=device),
]

# Define the problem to solve
question = "What is the sum of even numbers from 1 to 100?"

# Run the multi-agent debate
result = run_debate(agents, question)
print(f"Final decision: {result}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Agent1 (Solver):
To solve the problem, we need to identify all even numbers between 1 and 100. An even number is any number that is divisible by 2. So, we need to find all numbers in the range [1,100] that satisfy this condition.

## Step 1: Identify the range of even numbers
The range of numbers we are considering is from 1 to 100. To find the even numbers within this range, we need to consider every second number starting from 2 up to 100.

## Step 2: Determine the number of even numbers
Since we are considering every second number, the total count of even numbers in the range [1,100] can be found by dividing the last number (100) by 2 and rounding down to the nearest whole number, which gives us 50 even numbers.

## Step 3: Find the sum of an arithmetic series
The sum of an arithmetic series is given by the formula: sum = (n/2) * (first term + last term), where n is the number of terms. In this case, the first term is 2 (the first even number in the range), the last term is 100 (the

In [ ]:
First Round (Independent Responses):

In the first round, each agent (Solver, Critic, and Judge) independently provides their initial answer to the problem without relying on the others.
Subsequent Rounds (Refinement):

In subsequent rounds, each agent refines their response by considering the critiques and solutions provided by other agents in previous rounds. This encourages the agents to converge on a more accurate final answer.
Multiple Agents for the Same Role:

To closely follow the debate model from the paper, you can introduce multiple solvers and critics, allowing more diverse viewpoints and feedback during the debate process.

In [ ]:
First Round (Independent Responses):

In the first round, each agent (Solver, Critic, and Judge) independently provides their initial answer to the problem without relying on the others.
Subsequent Rounds (Refinement):

In subsequent rounds, each agent refines their response by considering the critiques and solutions provided by other agents in previous rounds. This encourages the agents to converge on a more accurate final answer.
Multiple Agents for the Same Role:

To closely follow the debate model from the paper, you can introduce multiple solvers and critics, allowing more diverse viewpoints and feedback during the debate process.

- First Round (Independent Responses):

In the first round, each agent (Solver, Critic, and Judge) independently provides their initial answer to the problem without relying on the others.

- Subsequent Rounds (Refinement):

In subsequent rounds, each agent refines their response by considering the critiques and solutions provided by other agents in previous rounds. This encourages the agents to converge on a more accurate final answer.

- Multiple Agents for the Same Role:

To closely follow the debate model from the paper, you can introduce multiple solvers and critics, allowing more diverse viewpoints and feedback during the debate process.

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import logging

logging.basicConfig(level=logging.INFO)

class Agent:
    def __init__(self, name, role, model_name=None, model=None, tokenizer=None, device=None, temperature=0.7, top_p=0.9, max_new_tokens=512):
        self.name = name
        self.role = role  # Solver, Critic, or Judge
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.max_new_tokens = max_new_tokens
        self.temperature = temperature
        self.top_p = top_p

        if model is not None and tokenizer is not None:
            # Use the provided model and tokenizer
            self.model = model.to(self.device)
            self.tokenizer = tokenizer
        elif model_name is not None:
            # Load tokenizer and model with error handling
            try:
                self.tokenizer = AutoTokenizer.from_pretrained(model_name)
                self.model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    torch_dtype=torch.float16 if self.device.type == 'cuda' else torch.float32
                ).to(self.device)
            except Exception as e:
                logging.error(f"Failed to load model {model_name}: {e}")
                raise
        else:
            raise ValueError("Either model and tokenizer or model_name must be provided")

        # Set up the pipeline with flexible parameters
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if self.device.type == 'cuda' else -1,
            max_new_tokens=self.max_new_tokens,
            do_sample=True,
            temperature=self.temperature,
            top_p=self.top_p,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
        )
        self.history = []
        self.score = 1.0

    def generate_response(self, messages):
        """
        Generate a response from the model based on the input messages.
        """
        prompt = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])
        prompt += f"\n{self.role}:"
        try:
            response = self.pipeline(prompt)[0]['generated_text']
            # Extract the assistant's response
            response = response[len(prompt):].strip()
        except Exception as e:
            logging.error(f"Error generating response: {e}")
            return "Sorry, I couldn't generate a response."
        return response


def construct_message(agent, previous_responses, question, round_num=1):
    """
    Construct a message for the agent based on its role and round of debate.
    """
    if agent.role == 'Solver':
        if round_num == 1:
            # In the first round, Solver provides an independent solution
            return [
                {'role': 'system', 'content': f"You are {agent.name}, a Solver. Provide a detailed solution to the problem."},
                {'role': 'user', 'content': f"The problem to solve is: '{question}'. Please provide your detailed solution."}
            ]
        else:
            # In subsequent rounds, Solver refines their solution based on previous critiques
            responses_summary = "\n".join([f"Critic's critique: {resp}" for resp in previous_responses])
            return [
                {'role': 'system', 'content': f"You are {agent.name}, a Solver. Revise your solution based on the feedback provided by the Critic."},
                {'role': 'user', 'content': f"The Critic provided the following feedback:\n{responses_summary}\nPlease refine your solution accordingly."}
            ]
    elif agent.role == 'Critic':
        # Critic critiques the Solver's solution
        responses_summary = "\n".join([f"Solver's solution: {resp}" for resp in previous_responses])
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Critic. Critique the solution provided by the Solver."},
            {'role': 'user', 'content': f"The Solver provided the following solution:\n{responses_summary}\nProvide your detailed critique."}
        ]
    elif agent.role == 'Judge':
        # Judge evaluates both the solution and the critique
        solver_response, critic_response = previous_responses
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Judge. Evaluate the solution and critique provided."},
            {'role': 'user', 'content': f"Solver's solution:\n{solver_response}\n\nCritic's critique:\n{critic_response}\nProvide your detailed evaluation and final decision."}
        ]


def run_debate(agents, question, rounds=3):
    """
    Run a multi-agent debate where agents respond in multiple rounds.
    """
    for round_num in range(1, rounds + 1):
        print(f"--- Round {round_num} ---")
        
        solver = next(agent for agent in agents if agent.role == 'Solver')
        critic = next(agent for agent in agents if agent.role == 'Critic')
        judge = next(agent for agent in agents if agent.role == 'Judge')

        # Step 1: Solver provides or refines the solution
        solver_messages = construct_message(solver, [], question, round_num)
        solver_response = solver.generate_response(solver_messages)
        print(f"{solver.name} (Solver):\n{solver_response}\n")

        # Step 2: Critic critiques the solution
        critic_messages = construct_message(critic, [solver_response], question)
        critic_response = critic.generate_response(critic_messages)
        print(f"{critic.name} (Critic):\n{critic_response}\n")

        # Step 3: Judge evaluates the final responses (only after the last round)
        if round_num == rounds:
            judge_messages = construct_message(judge, [solver_response, critic_response], question)
            judge_response = judge.generate_response(judge_messages)
            print(f"{judge.name} (Judge):\n{judge_response}\n")

    return judge_response  # Return final decision after all rounds


# Usage example
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model name
model_name = 'meta-llama/Llama-3.1-8B-Instruct'

# Load the model and tokenizer once
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32
    ).to(device)
except Exception as e:
    logging.error(f"Failed to load model {model_name}: {e}")
    raise

# Create agents with the same model and tokenizer
agents = [
    Agent(name='Agent1', role='Solver', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent2', role='Critic', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent3', role='Judge', model=model, tokenizer=tokenizer, device=device),
]

# Define the problem to solve
question = "What is the sum of even numbers from 1 to 100?"

# Run the multi-agent debate
result = run_debate(agents, question, rounds=3)
print(f"Final decision: {result}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

--- Round 1 ---
Agent1 (Solver):
To solve this problem, I will follow these steps:
Step 1: Identify the range of numbers (1 to 100) and understand the pattern of even numbers within this range.
Step 2: Determine the number of even numbers within the given range.
Step 3: Calculate the average of even numbers within the given range.
Step 4: Multiply the average by the total number of even numbers to get the sum.
Step 5: Verify the answer.

Step 1: Identify the range of numbers (1 to 100) and understand the pattern of even numbers within this range.
The even numbers from 1 to 100 are: 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100.

Step 2: Determine the number of even numbers within the given range.
There are 50 even numbers within the given range (1 to 100).

Step 3: Calculate the average of even numbers within the given range.
The average 

- Initial Round - Independent Solutions:

In the first round, multiple agents provide independent solutions (like the Solver role).

- Subsequent Rounds - Critique and Refinement:

In later rounds, agents refine their answers based on the critiques and solutions provided by other agents.

- Judge Role:

The Judge will step in only after multiple rounds have been completed to provide a final evaluation, rather than after every round.

In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import logging

logging.basicConfig(level=logging.INFO)

class Agent:
    def __init__(self, name, role, model=None, tokenizer=None, device=None, temperature=0.7, top_p=0.9, max_new_tokens=512):
        self.name = name
        self.role = role  # Solver, Critic, or Judge
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.max_new_tokens = max_new_tokens
        self.temperature = temperature
        self.top_p = top_p

        # Use the provided model and tokenizer
        self.model = model.to(self.device)
        self.tokenizer = tokenizer

        # Set up the pipeline with flexible parameters
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if self.device.type == 'cuda' else -1,
            max_new_tokens=self.max_new_tokens,
            do_sample=True,
            temperature=self.temperature,
            top_p=self.top_p,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
        )
        self.history = []

    def generate_response(self, messages):
        """
        Generate a response from the model based on the input messages.
        """
        prompt = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])
        prompt += f"\n{self.role}:"
        try:
            response = self.pipeline(prompt)[0]['generated_text']
            # Extract the assistant's response
            response = response[len(prompt):].strip()
        except Exception as e:
            logging.error(f"Error generating response: {e}")
            return "Sorry, I couldn't generate a response."
        return response

def construct_message(agent, previous_responses, question):
    """
    Construct a message for the agent based on its role.
    """
    if agent.role == 'Solver':
        # Solver provides independent solutions in the first round
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Solver. Solve the problem independently and in detail."},
            {'role': 'user', 'content': f"The problem to solve is: '{question}'. Please provide your solution before others respond."}
        ]
    elif agent.role == 'Critic':
        # Critic provides detailed feedback in subsequent rounds
        responses_summary = "\n".join([f"Other agent's solution: {resp}" for resp in previous_responses])
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Critic. Evaluate and critique the solutions provided by other agents. Suggest improvements."},
            {'role': 'user', 'content': f"The other agents have presented the following solutions:\n{responses_summary}\nProvide your critique and suggestions for improvement."}
        ]
    elif agent.role == 'Judge':
        # Judge evaluates and delivers a final verdict after all rounds
        solver_responses = "\n".join([f"Solution: {resp}" for resp in previous_responses])
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Judge. Evaluate all the solutions and critiques provided by other agents, and deliver a final decision."},
            {'role': 'user', 'content': f"The agents have provided the following solutions and critiques:\n{solver_responses}\nProvide your evaluation and final decision."}
        ]

def run_debate(agents, question, rounds=3):
    """
    Run a multi-agent debate where agents respond in multiple rounds.
    """
    all_responses = []  # Store responses from each round

    # Round 1: Each agent provides an independent solution
    for agent in agents:
        if agent.role == 'Solver':
            solver_messages = construct_message(agent, [], question)
            solver_response = agent.generate_response(solver_messages)
            all_responses.append(solver_response)
            print(f"{agent.name} (Solver):\n{solver_response}\n")

    # Rounds 2+: Each agent refines based on other agents' solutions
    for round_num in range(2, rounds + 1):
        print(f"--- Round {round_num} ---")
        for agent in agents:
            if agent.role == 'Critic':  # Critic role steps in to review all previous responses
                critic_messages = construct_message(agent, all_responses, question)
                critic_response = agent.generate_response(critic_messages)
                all_responses.append(critic_response)
                print(f"{agent.name} (Critic):\n{critic_response}\n")

            elif agent.role == 'Solver':  # Solver refines their solution based on critiques
                solver_messages = construct_message(agent, all_responses, question)
                solver_response = agent.generate_response(solver_messages)
                all_responses.append(solver_response)
                print(f"{agent.name} (Solver):\n{solver_response}\n")

    # Final Step: Judge evaluates after all rounds
    judge = next(agent for agent in agents if agent.role == 'Judge')
    judge_messages = construct_message(judge, all_responses, question)
    judge_response = judge.generate_response(judge_messages)
    print(f"{judge.name} (Judge):\n{judge_response}\n")

    return judge_response

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model name
model_name = 'meta-llama/Llama-3.1-8B-Instruct'

# Load the model and tokenizer once
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32
    ).to(device)
except Exception as e:
    logging.error(f"Failed to load model {model_name}: {e}")
    raise

# Create agents with the same model and tokenizer
agents = [
    Agent(name='Agent1', role='Solver', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent2', role='Solver', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent3', role='Critic', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent4', role='Judge', model=model, tokenizer=tokenizer, device=device),
]

# Define the problem to solve
question = "What is the sum of even numbers from 1 to 100?"

# Run the multi-agent debate
result = run_debate(agents, question)
print(f"Final decision: {result}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Agent1 (Solver):
To find the sum of even numbers from 1 to 100, I will follow these steps:

1. List the even numbers from 1 to 100: 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100.

2. Identify the pattern: The even numbers form an arithmetic sequence with a common difference of 2.

3. Find the number of terms: Since the sequence starts from 2 and ends at 100, with a common difference of 2, the number of terms can be found using the formula for the nth term of an arithmetic sequence: a_n = a_1 + (n-1)d, where a_n is the nth term, a_1 is the first term, n is the number of terms, and d is the common difference. Solving for n, we get: 100 = 2 + (n-1)2. Simplifying, we get: 98 = (n-1)2. Dividing both sides by 2, we get: 49 = n-1. Adding 1 to both sides, we get: n = 50.

4. Find the sum: The sum of an arithmetic sequence can be found using the f

# Which is Better?

## First Version (Simpler, Sequential, Role-Specific):
- Best for: If you want a clear and structured debate where roles are well-defined and each agent has a unique responsibility, this version is better. It’s modular, easy to extend, and straightforward to follow. It fits well for scenarios where each agent specializes in a specific task, and there is a clear flow from solving to critiquing to judging.
- Ideal Use Case: If you want to start with a more structured and deterministic approach where the debate evolves in a controlled manner.
    
## Second Version (Flexible, Multiple Solvers, Dynamic):
- Best for: If you want more diversity in the debate, where multiple Solvers can independently propose solutions and receive critiques, this version is better. It’s more flexible and can scale easily with more agents of the same role.
- Ideal Use Case: If your goal is to simulate more complex debates where multiple agents propose competing solutions and critiques, leading to a richer exchange of ideas.
    
## Conclusion:
If you want simplicity, clear role definitions, and easier maintenance, go with the First Version.
If you need more flexibility and diversity of ideas (with multiple agents of the same role), the Second Version is better.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import logging
import json
import numpy as np
import re

logging.basicConfig(level=logging.INFO)

class Agent:
    def __init__(self, name, role, model=None, tokenizer=None, device=None, temperature=0.7, top_p=0.9, max_new_tokens=256):
        self.name = name
        self.role = role  # Solver, Critic, or Judge
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.max_new_tokens = max_new_tokens
        self.temperature = temperature
        self.top_p = top_p

        # Use the provided model and tokenizer
        self.model = model.to(self.device)
        self.tokenizer = tokenizer

        # Set up the pipeline with flexible parameters
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if self.device.type == 'cuda' else -1,
            max_new_tokens=self.max_new_tokens,
            do_sample=True,
            temperature=self.temperature,
            top_p=self.top_p,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
        )
        self.history = []

    def generate_response(self, messages):
        """
        Generate a response from the model based on the input messages.
        """
        prompt = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])
        prompt += f"\n{self.role}:"
        try:
            response = self.pipeline(prompt)[0]['generated_text']
            # Extract the assistant's response
            response = response[len(prompt):].strip()
        except Exception as e:
            logging.error(f"Error generating response: {e}")
            return "Sorry, I couldn't generate a response."
        return response

def construct_message(agent, previous_responses, question, options):
    """
    Construct a message for the agent based on its role.
    """
    # Format options properly
    formatted_options = "\n".join([f"({chr(65 + i)}) {option.strip()}" for i, option in enumerate(options.split(", "))])

    if agent.role == 'Solver':
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Solver. Solve the problem and select the correct answer from the options provided: (A), (B), (C), or (D)."},
            {'role': 'user', 'content': f"The problem to solve is: '{question}'. Here are the options:\n{formatted_options}\nPlease provide the correct answer in the form of (A), (B), (C), or (D), and explain briefly."}
        ]

    elif agent.role == 'Critic':
        responses_summary = "\n".join([f"Solver's solution: {resp}" for resp in previous_responses])
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Critic. Evaluate the solutions provided by the Solvers and select the correct answer from the options provided."},
            {'role': 'user', 'content': f"The Solvers have presented the following solutions:\n{responses_summary}\nHere are the options:\n{formatted_options}\nPlease provide your critique and select the correct answer from the available options (A), (B), (C), or (D)."}
        ]

    elif agent.role == 'Judge':
        solver_responses = "\n".join([f"Solution: {resp}" for resp in previous_responses])
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Judge. Evaluate all the solutions and critiques provided by the Solvers and the Critic, and deliver a final decision."},
            {'role': 'user', 'content': f"The agents have provided the following solutions and critiques:\n{solver_responses}\nHere are the options:\n{formatted_options}\nPlease provide your final decision, and select the correct answer in the form of (A), (B), (C), or (D)."}
        ]

def run_debate(agents, question, options, rounds=3):
    """
    Run a multi-agent debate where agents respond in multiple rounds.
    """
    all_responses = []  # Store responses from each round

    # Round 1: Each agent provides an independent solution
    for agent in agents:
        if agent.role == 'Solver':
            solver_messages = construct_message(agent, [], question, options)
            solver_response = agent.generate_response(solver_messages)
            all_responses.append(solver_response)
            print(f"{agent.name} (Solver):\n{solver_response}\n")

    # Rounds 2+: Each agent refines based on other agents' solutions
    for round_num in range(2, rounds + 1):
        print(f"--- Round {round_num} ---")
        for agent in agents:
            if agent.role == 'Critic':  # Critic role steps in to review all previous responses
                critic_messages = construct_message(agent, all_responses, question, options)
                critic_response = agent.generate_response(critic_messages)
                all_responses.append(critic_response)
                print(f"{agent.name} (Critic):\n{critic_response}\n")

            elif agent.role == 'Solver':  # Solver refines their solution based on critiques
                solver_messages = construct_message(agent, all_responses, question, options)
                solver_response = agent.generate_response(solver_messages)
                all_responses.append(solver_response)
                print(f"{agent.name} (Solver):\n{solver_response}\n")

    # Final Step: Judge evaluates after all rounds
    judge = next(agent for agent in agents if agent.role == 'Judge')
    judge_messages = construct_message(judge, all_responses, question, options)
    judge_response = judge.generate_response(judge_messages)
    print(f"{judge.name} (Judge):\n{judge_response}\n")

    return judge_response

def parse_answer(input_str):
    """
    Parse the model's output to extract the predicted multiple-choice answer (A, B, C, etc.).
    Ensures the returned answer is wrapped in parentheses.
    """
    pattern = r'\((\w)\)'  # Look for patterns like (A), (B), etc.
    matches = re.findall(pattern, input_str)
    if matches:
        return f"({matches[0].upper()})"  # Return the first matched answer in parentheses
    return None

def evaluate_on_mmlu(agents, mmlu_data):
    """
    Evaluate the multi-agent debate system on MMLU data and compute accuracy.
    Ensures the predicted answer is always in parentheses.
    """
    accuracies = []
    for entry in mmlu_data:
        question = entry['question']
        options = entry['options']
        correct_answer = entry['answer']  # Expected answer like (A), (B), etc.

        print(f"Evaluating question: {question}")

        # Run the debate system to get the final decision
        final_judge_decision = run_debate(agents, question, options, rounds=3)

        # Extract the predicted answer from the Judge's decision
        predicted_answer = parse_answer(final_judge_decision)

        # Check if the predicted answer matches the correct answer
        accurate = 1 if predicted_answer == correct_answer else 0
        accuracies.append(accurate)

        # Ensure the predicted answer is in parentheses
        predicted_answer_formatted = predicted_answer if predicted_answer else "None"
        print(f"Predicted: {predicted_answer_formatted}, Correct: {correct_answer}, Accuracy: {accurate}")

    # Compute overall accuracy
    mean_accuracy = np.mean(accuracies)
    print(f"Mean Accuracy: {mean_accuracy}")
    return mean_accuracy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model name
model_name = 'meta-llama/Llama-3.1-8B-Instruct'

# Load the model and tokenizer once
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32
    ).to(device)
except Exception as e:
    logging.error(f"Failed to load model {model_name}: {e}")
    raise

# Create agents with the same model and tokenizer
agents = [
    Agent(name='Agent1', role='Solver', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent2', role='Solver', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent3', role='Solver', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent4', role='Critic', model=model, tokenizer=tokenizer, device=device),
    Agent(name='Agent5', role='Judge', model=model, tokenizer=tokenizer, device=device),
]

# Load MMLU data (assumed to be in JSON format)
with open("gsm8k_small.json", "r") as file:
    mmlu_data = json.load(file)

# Run evaluation
evaluate_on_mmlu(agents, mmlu_data)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Agent1 (Solver):
To find the total number of clips Natalia sold in April and May, we need to calculate the number of clips she sold in each month and then add them together. In April, Natalia sold clips to 48 of her friends. In May, she sold half as many clips, which is 48 / 2 = 24 clips. To find the total number of clips sold in both months, we add 48 (April) and 24 (May), which equals 72. The correct answer is (A). 

The final answer is (A).  ## Step 1: Determine the number of clips sold in April.
Natalia sold clips to 48 of her friends in April.

## Step 2: Calculate the number of clips sold in May.
Natalia sold half as many clips in May as she did in April, so she sold 48 / 2 = 24 clips in May.

## Step 3: Add the number of clips sold in April and May to find the total.
To find the total number of clips sold

In [ ]:
!pip install numpy==1.22.4 openai==0.27.6 pandas==1.5.3 tqdm==4.64.1 -q

In [32]:
from glob import glob
import pandas as pd
import json
import time
import random
import openai

def construct_message(agents, question, idx):
    if len(agents) == 0:
        return {"role": "user", "content": "Can you double check that your answer is correct. Put your final answer in the form (X) at the end of your response."}

    prefix_string = "These are the solutions to the problem from other agents: "

    for agent in agents:
        agent_response = agent[idx]["content"]
        response = "\n\n One agent solution: ```{}```".format(agent_response)

        prefix_string = prefix_string + response

    prefix_string = prefix_string + """\n\n Using the reasoning from other agents as additional advice, can you give an updated answer? Examine your solution and that other agents step by step. Put your answer in the form (X) at the end of your response.""".format(question)
    return {"role": "user", "content": prefix_string}


def construct_assistant_message(completion):
    content = completion["choices"][0]["message"]["content"]
    return {"role": "assistant", "content": content}


def generate_answer(answer_context):
    try:
        completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo-0301",
                  messages=answer_context,
                  n=1)
    except:
        print("retrying due to an error......")
        time.sleep(20)
        return generate_answer(answer_context)

    return completion


def parse_question_answer(df, ix):
    question = df.iloc[ix, 0]
    a = df.iloc[ix, 1]
    b = df.iloc[ix, 2]
    c = df.iloc[ix, 3]
    d = df.iloc[ix, 4]

    question = "Can you answer the following question as accurately as possible? {}: A) {}, B) {}, C) {}, D) {} Explain your answer, putting the answer in the form (X) at the end of your response.".format(question, a, b, c, d)

    answer = df.iloc[ix, 5]

    return question, answer


agents = 3
rounds = 2

tasks = glob("/data/vision/billf/scratch/yilundu/llm_iterative_debate/mmlu/data/test/*.csv")

dfs = [pd.read_csv(task) for task in tasks]

random.seed(0)
response_dict = {}

for i in range(100):
    df = random.choice(dfs)
    ix = len(df)
    idx = random.randint(0, ix-1)

    question, answer = parse_question_answer(df, idx)

    agent_contexts = [[{"role": "user", "content": question}] for agent in range(agents)]

    for round in range(rounds):
        for i, agent_context in enumerate(agent_contexts):

            if round != 0:
                agent_contexts_other = agent_contexts[:i] + agent_contexts[i+1:]
                message = construct_message(agent_contexts_other, question, 2 * round - 1)
                agent_context.append(message)

            completion = generate_answer(agent_context)

            assistant_message = construct_assistant_message(completion)
            agent_context.append(assistant_message)
            print(completion)

    response_dict[question] = (agent_contexts, answer)

json.dump(response_dict, open("mmlu_{}_{}.json".format(agents, rounds), "w"))

IndexError: list index out of range